## Install Dependencies

In [ ]:
# Install necessary packages with specific versions
!pip install tokenizers
!pip install transformers==4.49.0
!pip install unsloth==2025.3.6
!pip install unsloth_zoo==2025.3.4
!pip install unstructured
!pip install vllm==0.7.3

# Loading MIMIC-IV Data into DuckDB

This code connects to an in-memory DuckDB instance and loads CSV files from the MIMIC-IV dataset into separate DuckDB tables. This setup allows fast and efficient querying of structured medical data without needing a full database server. You will need access to 'MIMIC-IV-Ext Clinical Decision Making: A MIMIC-IV Derived Dataset for Evaluation of Large Language Models on the Task of Clinical Decision Making for Abdominal Pathologies' and need to have the data loaded in your environment for this to work.


In [1]:
import duckdb

# Connect to an in-memory DuckDB instance
con = duckdb.connect(database=':memory:')

# Define paths and corresponding table names
csv_table_map = {
    'lab_test_mapping': '/content/physionet.org/files/mimic-iv-ext-cdm/1.1/lab_test_mapping.csv',
    'laboratory_tests': '/content/physionet.org/files/mimic-iv-ext-cdm/1.1/laboratory_tests.csv',
    'microbiology': '/content/physionet.org/files/mimic-iv-ext-cdm/1.1/microbiology.csv',
    'physical_examination': '/content/physionet.org/files/mimic-iv-ext-cdm/1.1/physical_examination.csv',
    'radiology_reports': '/content/physionet.org/files/mimic-iv-ext-cdm/1.1/radiology_reports.csv',
}

# Load each CSV into DuckDB
for table_name, csv_path in csv_table_map.items():
    con.execute(f"""
        CREATE TABLE {table_name} AS
        SELECT * FROM read_csv_auto('{csv_path}');
    """)
    print(f"Loaded table: {table_name}")

Loaded table: lab_test_mapping
Loaded table: laboratory_tests
Loaded table: microbiology
Loaded table: physical_examination
Loaded table: radiology_reports


# DuckDB Query Helper Function

This function executes an SQL query against the DuckDB database and returns the results as a JSON-formatted string. If the query fails or returns no data, it safely returns `"no data"` instead.


In [2]:
import pandas as pd
import json

def query_duckdb_as_json(query: str) -> str:
    try:
        df = con.execute(query).fetchdf()
        if df.empty:
            return "no data"
        return json.dumps(df.to_dict(orient='records'), indent=2)
    except Exception as e:
        return "no data"


# Preparing the Clinical Dataset

This section loads the history of present illness (HPI) notes and associated pathology labels.
It formats the dataset into a conversational prompt format, applying a system prompt that
instructs the model on how to query or diagnose. The data is then converted into a Hugging Face
Dataset and split into training (90%) and validation (10%) sets.


In [3]:
import pandas as pd
import json
from datasets import Dataset

# Load the HPI notes
hpi_df = pd.read_csv("/content/physionet.org/files/mimic-iv-ext-cdm/1.1/history_of_present_illness.csv")

# Load the pathology labels
with open("/content/physionet.org/files/mimic-iv-ext-cdm/1.1/pathology_ids.json", "r") as f:
    pathology_ids = json.load(f)

# Reverse-map hadm_id to disease label
hadm_to_label = {}
for disease, ids in pathology_ids.items():
    for hadm_id in ids:
        hadm_to_label[hadm_id] = disease

# Filter only those rows in the HPI dataframe that have a known label
labeled_df = hpi_df[hpi_df['hadm_id'].isin(hadm_to_label.keys())].copy()

# Add the label column
labeled_df['label'] = labeled_df['hadm_id'].map(hadm_to_label)

# This is your system prompt
SYSTEM_PROMPT = """
You are a medical assistant who can query clinical data to aid in diagnosing patients with one of four diseases:
appendicitis, cholecystitis, pancreatitis, or diverticulitis.

You will be given:
- A `hadm_id` (hospital admission ID)
- A `history_of_present_illness` (HPI) note

From this, you may:
- Attempt to diagnose the patient directly
- Issue SQL queries to any of the following tables: `laboratory_tests`, `microbiology`, `physical_examination`, `radiology_reports`, `lab_test_mapping`

To issue a query, wrap your SQL statement in `<search>...</search>` tags.

For example:
<search>SELECT * FROM radiology_reports WHERE hadm_id = 1089609</search>

You can use the `lab_test_mapping` table to look up an `itemid` from `laboratory_tests` if you want more information about what the test refers to.

For example:
<search>SELECT * FROM lab_test_mapping WHERE itemid = 99882</search>

The result of your query will be returned in `<information>...</information>` tags.
If no data is available for your query, the system will respond with:
<information>no data</information>

Before giving a diagnosis:
- Explain your reasoning using `<think>...</think>` tags
- Then provide **one** of the four possible diagnoses using `<answer>...</answer>` tags
"""

# Format the dataset
def format_example(row):
    user_message = f"hadm_id: {row['hadm_id']} \nhpi: {row['hpi']}"
    return {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': user_message}
        ],
        'answer': row['label']
    }

# Convert the DataFrame to a list of dicts
examples = labeled_df.apply(format_example, axis=1).tolist()

# Create the HF Dataset
final_dataset = Dataset.from_list(examples)
# Shuffle the dataset
final_dataset = final_dataset.shuffle(seed=42)

# Compute the split index for 90/10
split_index = int(0.9 * len(final_dataset))

# Perform the split
train_dataset = final_dataset.select(range(split_index))
val_dataset = final_dataset.select(range(split_index, len(final_dataset)))

# Loading and Preparing the Model

This section loads the chosen model using Unsloth's FastLanguageModel.
It patches the model for Group Relative Policy Optimization (GRPO) and enables efficient fine-tuning
with Low-Rank Adaptation (LoRA) and 4-bit quantization for memory savings.
The model is also wrapped for parameter-efficient training with selective target modules and gradient checkpointing.


In [4]:
from unsloth import FastLanguageModel, PatchFastRL
from unsloth.chat_templates import get_chat_template

# Patch the FastLanguageModel to integrate GRPO-specific modifications.
PatchFastRL("GRPO", FastLanguageModel)

from unsloth import is_bfloat16_supported
import torch

# Set maximum sequence length and LoRA rank (controls the adaptation complexity).
max_seq_length = 1024*3  # Increase if you need longer reasoning traces.
lora_rank = 64         # Larger rank can improve performance but may slow down training.

# Load the model in 4-bit mode for reduced memory usage and enable fast inference with vLLM.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "HuggingFaceTB/SmolLM2-360M-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True,           # Set to False if using LoRA in 16-bit precision.
    fast_inference = True,         # Enable vLLM for faster inference.
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5,  # Adjust GPU memory usage to avoid out-of-memory errors.
)

#tokenizer = get_chat_template(tokenizer, chat_template="qwen2.5")

# Wrap the model with PEFT (Parameter-Efficient Fine-Tuning) using LoRA.
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,           # Use a rank greater than 0
    lora_alpha = lora_rank,  # A higher lora_alpha value means that the LoRA layers have a greater influence on the model's output,
                             # while a lower value reduces this influence
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],                                       # Specify target modules; you can remove QKVO if memory is limited.
    use_gradient_checkpointing = "unsloth",  # Enable gradient checkpointing for long context finetuning.
    random_state = 3407,                     # Set a random seed for reproducibility.
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-24 19:11:19 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.6: Fast Llama patching. Transformers: 4.49.0. vLLM: 0.7.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading HuggingFaceTB/SmolLM2-360M-Instruct with actual GPU utilization = 49.43%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.56 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 3072. Num Sequences = 288.
Unsloth: vLLM's KV Cache can use up to 18.81 GB.

tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

HuggingFaceTB/SmolLM2-360M-Instruct does not have a padding token! Will use pad_token = <|endoftext|>.


Unsloth 2025.3.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Reward Function Definitions

This section defines the custom reward functions used during reinforcement learning:

- **`information_reward_func`**: Rewards the model +1 for each valid SQL query that returns data. Ignores queries that were only examples shown in the system prompt.
- **`thinking_reward_func`**: Rewards the model +1 for including `<think>...</think>` tags, encouraging it to explain its reasoning. Ignores trivial example tags.
- **`diagnosis_reward_func_silent`**: Rewards the model +2 if it correctly predicts the diagnosis within `<answer>...</answer>` tags. Otherwise, gives 0 points for wrong or missing answers, also ignoring trivial answers.


In [5]:
import re

def extract_xml_answer(text: str) -> str:
    """Extracts content inside <answer>...</answer> tags."""
    match = re.search(r"<answer>(.*?)</answer>", text, re.DOTALL)
    return match.group(1).strip() if match else ""

def diagnosis_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    """
    Reward logic:
    - +2 if model gives a correct answer with <answer> tags (excluding templated examples).
    - -1 if it gives a wrong answer with <answer> tags.
    -  0 if it gives no useful answer.
    """
    responses = [completion[0]['content'] for completion in completions]
    extracted_answers = [extract_xml_answer(r) for r in responses]
    gold_answers = [a.lower().strip() for a in answer]

    # Any known template answers to ignore (e.g., "Yes" from template, or empty filler)
    ignored_answers = {"yes", "no", "", "..."}

    rewards = []

    for i, (prompt, full_response, extracted, gold) in enumerate(zip(prompts, responses, extracted_answers, gold_answers)):
        user_prompt = prompt[-1]['content'] if isinstance(prompt, list) else prompt
        print("=" * 40)
        print(f"Example {i + 1}")
        print("User Input:")
        print(user_prompt)
        print("Gold Answer:")
        print(gold)
        print("Model Response:")
        print(full_response)

        extracted_clean = extracted.lower().strip()

        if extracted_clean in ignored_answers:
            rewards.append(0.0)
        elif extracted_clean == gold:
            rewards.append(2.0)
        else:
            rewards.append(-1.0)

    return rewards

def extract_search_queries(text: str) -> list[str]:
    """Extracts all content inside <search>...</search> tags."""
    return re.findall(r"<search>(.*?)</search>", text, re.DOTALL)

def information_reward_func(completions, **kwargs) -> list[float]:
    example_queries = {
        "SELECT * FROM radiology_reports WHERE hadm_id = 1089609",
        "SELECT * FROM lab_test_mapping WHERE itemid = 99882",
        "..."
    }

    rewards = []

    for completion in completions:
        text = completion[0]['content']
        queries = extract_search_queries(text)

        if not queries:
            rewards.append(0.0)
            continue

        score = 0.0
        for q in queries:
            cleaned_query = q.strip()

            # Skip example queries from the system prompt
            if cleaned_query in example_queries:
                continue

            result = query_duckdb_as_json(cleaned_query)

            if result != "no data":
                score += 1.0

        rewards.append(score)

    return rewards


def thinking_reward_func(completions, **kwargs) -> list[float]:
    """
    Rewards the model for including <think>...</think> tags,
    but ignores the literal example <think>...</think> from the system prompt.
    - +1 if real <think> tags are present.
    - 0 if no <think> tags are found (excluding the prompt example).
    """
    rewards = []

    for completion in completions:
        text = completion[0]['content']

        # Find all think tags
        think_matches = re.findall(r"<think>(.*?)</think>", text, re.DOTALL)
        think_matches = [t.strip() for t in think_matches]

        # Filter out the exact example tag content
        real_thinks = [t for t in think_matches if t != "..."]

        rewards.append(1.0 if real_thinks else 0.0)

    return rewards


def diagnosis_reward_func_silent(prompts, completions, answer, **kwargs) -> list[float]:
    """
    Updated version for training:
    - +2 if correct answer with <answer> tags (excluding examples)
    -  0 otherwise (either wrong answer or no answer)
    """
    responses = [completion[0]['content'] for completion in completions]
    extracted_answers = [extract_xml_answer(r) for r in responses]
    gold_answers = [a.lower().strip() for a in answer]

    ignored_answers = {"yes", "no", "..."}

    rewards = []
    for extracted, gold in zip(extracted_answers, gold_answers):
        cleaned = extracted.lower().strip()
        if cleaned == "" or cleaned in ignored_answers:
            rewards.append(0.0)
        elif cleaned == gold:
            rewards.append(2.0)
        else:
            rewards.append(0.0)  # wrong answer also gets 0

    return rewards


# Preview and Save Model Conversations

This function generates and previews sample prompt-response pairs from the model.  
It prints each conversation to the console and also saves it to a text file for later presentation.

- `model`: The Hugging Face model to evaluate.
- `tokenizer`: The tokenizer for formatting inputs.
- `dataset`: Dataset containing prompts and answers.
- `file_path`: File where outputs are saved.
- `num_examples`: Number of examples to preview.
- `max_new_tokens`: Maximum number of tokens generated per prompt.


In [6]:
def preview_and_save_model_conversations(model, tokenizer, dataset, file_path="model_outputs.txt", num_examples=10, max_new_tokens=512):
    """
    Prints and saves prompt-response pairs from the dataset for presentation.

    Args:
        model: Hugging Face model.
        tokenizer: Corresponding tokenizer.
        dataset: Hugging Face Dataset.
        file_path: Output text file path.
        num_examples: How many examples to preview.
        max_new_tokens: Max number of new tokens to generate.
    """
    model.eval()
    with open(file_path, "w", encoding="utf-8") as f:
        for i in range(num_examples):
            example = dataset[i]
            structured_prompt = example["prompt"]
            input_text = tokenizer.apply_chat_template(structured_prompt, tokenize=False, add_generation_prompt=True)
            input_ids = tokenizer(input_text, return_tensors="pt", padding=True).input_ids.to(model.device)

            output_ids = model.generate(
                input_ids=input_ids,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

            generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
            prompt_only = tokenizer.decode(input_ids[0], skip_special_tokens=True)
            completion = generated_text[len(prompt_only):].strip()

            separator = "=" * 50
            output_block = (
                f"{separator}\n"
                f"Example {i+1}\n\n"
                f"User Prompt:\n{input_text}\n\n"
                f"Model Response:\n{completion}\n"
                f"{separator}\n\n"
            )

            # Print to console
            print(output_block)

            # Write to file
            f.write(output_block)


# Observe How the Model Behaves Before Training

In [ ]:
preview_and_save_model_conversations(model, tokenizer, val_dataset.select(range(10)), file_path="before_training_SmolLM2-360M-Instruct.txt")

# Configure GRPO Training and Initialize Trainer

This section sets up the GRPO training configuration and initializes the GRPOTrainer.

- `GRPOConfig` defines all the training hyperparameters like learning rate, batch size, optimizer settings, and generation settings.
- `GRPOTrainer` is created using the model, tokenizer, reward functions, and training dataset.
- vLLM is enabled to accelerate inference during training.
- Training will run for a maximum of 250 steps with model checkpoints saved at the same interval.


In [8]:
from trl import GRPOConfig, GRPOTrainer

# Configure GRPO training parameters.
# This configuration sets up the training hyperparameters, optimization settings, and inference acceleration via vLLM.
training_args = GRPOConfig(
    use_vllm = True,                     # Enable vLLM to accelerate inference during training.
    learning_rate = 5e-6,                # Set the learning rate for the optimizer.
    adam_beta1 = 0.9,                    # First beta parameter for the AdamW optimizer.
    adam_beta2 = 0.99,                   # Second beta parameter for the AdamW optimizer.
    weight_decay = 0.1,                  # Weight decay to regularize the model and prevent overfitting.
    warmup_ratio = 0.1,                  # Fraction of steps used for learning rate warmup.
    lr_scheduler_type = "cosine",        # Use cosine annealing for the learning rate scheduler.
    optim = "adamw_8bit",                # Use 8-bit AdamW optimizer for memory efficiency.
    logging_steps = 1,                   # Log training information every step.
    bf16 = is_bfloat16_supported(),      # Use bfloat16 precision if supported by the GPU.
    fp16 = not is_bfloat16_supported(),  # Otherwise, fall back to fp16 precision.
    per_device_train_batch_size = 8,     # Batch size per device during training.
    gradient_accumulation_steps = 1,     # Accumulate gradients over this many steps (increase for smoother training if needed).
    num_generations = 8,                 # Number of generations per prompt (reduce if memory issues occur).
    max_prompt_length = 1024,            # Maximum length for the input prompt.
    max_completion_length = 512,         # Maximum length for the generated completion.
    num_train_epochs = 1,                # Uncomment this line to run training for one epoch.
    max_steps = 250,                     # Maximum number of training steps.
    save_steps = 250,                    # Save the model checkpoint every specified number of steps.
    max_grad_norm = 0.1,                 # Maximum gradient norm for gradient clipping.
    report_to = ["tensorboard"],         # Report to tensorboard to view training metrics.
    output_dir = "outputs",              # Directory to save the training outputs and checkpoints.
)

# Instantiate the GRPO trainer with the model, tokenizer, reward functions, and training dataset.
trainer = GRPOTrainer(
    model = model,                       # The language model to be trained.
    processing_class = tokenizer,        # The tokenizer used to preprocess the data.
    reward_funcs = [
        diagnosis_reward_func_silent,
        information_reward_func,
        thinking_reward_func
    ],
    args = training_args,                # GRPO training configuration.
    train_dataset = train_dataset,       # The training dataset containing prompts and expected answers.
)

# Start Training and Save the LoRA Model

This section launches the GRPO training process and saves the LoRA-adapted model after training:

- `trainer.train()` begins the reinforcement learning fine-tuning using GRPO.
- `model.save_lora("grpo_saved_lora")` saves the fine-tuned LoRA weights to disk for later loading and inference.


In [9]:
# Begin training using the GRPO algorithm.
trainer.train()

# Save the LoRA-adapted model for later use.
model.save_lora("grpo_saved_lora")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,160 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 34,734,080/396,555,200 (8.76% trained)


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / diagnosis_reward_func_silent,rewards / information_reward_func,rewards / thinking_reward_func
1,0.000000,1.750000,2.492847,265.250000,0.000000,0.000000,1.750000,0.000000
2,-0.000000,0.875000,1.246423,281.750000,0.000000,0.000000,0.625000,0.250000
3,0.000000,3.125000,3.226564,285.000000,0.000571,0.500000,2.500000,0.125000
4,0.000000,1.250000,1.488048,294.750000,0.000938,0.000000,1.250000,0.000000
5,0.000000,1.250000,0.707107,194.875000,0.000717,0.000000,0.875000,0.375000
6,0.000000,1.875000,2.232071,195.875000,0.000677,0.250000,1.625000,0.000000
7,0.000000,1.000000,1.414214,327.500000,0.000923,0.000000,0.875000,0.125000
8,0.000000,1.125000,0.640870,121.000000,0.000778,0.000000,0.750000,0.375000
9,0.000000,0.750000,0.886405,258.500000,0.000962,0.000000,0.375000,0.375000
10,0.000000,0.250000,0.707107,151.250000,0.000900,0.000000,0.125000,0.125000


Unsloth: Will smartly offload gradients to save VRAM!


# Launch TensorBoard to Monitor Training

This section loads TensorBoard and points it to the training logs directory:

- `%load_ext tensorboard` loads TensorBoard extension into the notebook.
- `%tensorboard --logdir /content/outputs/runs` starts TensorBoard to visualize training metrics like loss, rewards, etc.


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/outputs/runs

# Observe How the Model Behaves After Training

In [ ]:
preview_and_save_model_conversations(model, tokenizer, val_dataset.select(range(10)), file_path="after_training_SmolLM2-360M-Instruct.txt")

# Acknowledgements

Parts of this notebook, including GRPO configuration settings and LoRA fine-tuning parameters, are inspired by the following tutorials:

- [Hugging Face Fine-Tuning LLMs with GRPO (Cookbook)](https://huggingface.co/learn/cookbook/en/fine_tuning_llm_grpo_trl)
- [Kaggle: Fine-Tuning Qwen2.5-3B-Instruct with GRPO + PEFT](https://www.kaggle.com/code/ksmooi/fine-tuning-qwen2-5-3b-instruct-grpo-peft/notebook#Step-5:-Configuring-and-Running-the-Trainer)

Their resources were helpful in setting up this training pipeline.
